---
title : "Agent 활용 | Agentic RAG(retrieval, web search, tool calling)"
---
# Agentic RAG

이번엔 문서 검색을 통해 최신 정보에 근접하여 검색 결과를 가지고 답변을 생성하는 에이전트를 만들어보자.

질문에 따라 문서를 검색하여 답변하거나, 인터넷 검색 도구를 활용하여 답변하는 에이전트를 만들어 볼것이다.

Agent를 활용하여 RAG를 수행한다면 이를 Agentic RAG라고 한다.

## Tools

Agent가 활용할 도구를 정의하여 Agent가 추론(reasoning)을 수행할 때 활용하도록 만들 수 있다. 

Tavily Search 는 대표적인 인터넷 검색 도구이며, 검색을 통해 최신 정보를 얻어 답변을 생성할수 있다. 도구는 검색 뿐만아니라 Python코드를 실행할 수 있는 도구, 직접 정의한 함수를 실행하는 도구 등 다양한 종류와 방법론을 제공한다.

### Web Search Tool : Tavily Search
Langchain에는 Tavily 검색 엔진을 도구로 쉽게 사용할수 있는 내장 도구가 있다.

Tavily Search를 사용하기 위해서는 API key가 필요하다. 

- [Tavily API key 발급](https://app.tavily.com/home?code=qBjTyMCT1oPN5i7kReMKW9EhqDEmYgTkswdl_lze12AR_&state=eyJyZXR1cm5UbyI6Ii9ob21lIn0)

발급 받은 API KEY 를 `.env` 파일에 다음과 같이 등록한다.

`TAVILY_API_KEY=(발급 받은 Tavily API KEY)`